In [ ]:
from torchvision.datasets import STL10
from models.transforms import RandomPatchTexture,Blur,DistortedTexture,Outline,Silhouette, Grayscale, CleanerSilhouette
from torchvision import transforms
import numpy as np
import os
import cv2

In [ ]:
# Function to load binary files
def load_stl10_bin(file, num_images, image_size=96, num_channels=3):
    with open(file, 'rb') as f:
        images = np.fromfile(f, dtype=np.uint8)
        # Reshape to (num_images, num_channels, height, width)
        images = images.reshape((num_images, num_channels, image_size, image_size))
        # Reorder to (num_images, height, width, num_channels) to match typical image format
        for im in images:
            for i in range(num_channels):
                im[i] = np.rot90(im[i], k=-1)
        images = np.transpose(images, (0, 2, 3, 1))  # Convert to (num_images, H, W, C)
        # for i in range(num_channels):
        #     images[:,:,:,i] = np.rot90(images[:,:,i], k=-1)
    return images

# Load train and test datasets
train_images = load_stl10_bin('data/stl10_binary/train_X.bin', num_images=5000)
train_labels = np.fromfile('data/stl10_binary/train_y.bin', dtype=np.uint8)

test_images = load_stl10_bin('data/stl10_binary/test_X.bin', num_images=8000)
test_labels = np.fromfile('data/stl10_binary/test_y.bin', dtype=np.uint8)
train_labels = train_labels - 1
test_labels = test_labels - 1

### Basic Evaluation

In [ ]:
from models.model import ResNet50, ViTBase, CLIPModel
from models.utils import train_step, eval_step, DataLoaders, CustomDataset
from models.transforms import transforms_resnet,transforms_clip_vit, transforms_vit
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10, CIFAR100, SVHN
from tqdm import tqdm
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
BATCH_SIZE = 64
N_EPOCHS = 3

In [ ]:
def train_and_eval(train_loader,test_loader,model,loss_fn,optimizer,device,modeltype):
    tr_metric = {"Accuracy":[],"Loss":[]}
    ts_metric = {"Accuracy":[],"Loss":[]}

    for epoch in tqdm(range(N_EPOCHS)):
        tr_loss, tr_acc = train_step(model,train_loader,loss_fn,optimizer,device,modeltype,data='stl10')
        ts_loss, ts_acc = eval_step(model,test_loader,loss_fn,device,modeltype,data="stl10")

        tr_metric["Accuracy"].append(tr_acc)
        tr_metric["Loss"].append(tr_loss)

        ts_metric["Accuracy"].append(ts_acc)
        ts_metric["Loss"].append(ts_loss)
    
    return tr_metric, ts_metric

In [ ]:
train = CustomDataset(train_images,train_labels,transforms_resnet)
test = CustomDataset(test_images,test_labels,transforms_resnet)

dl = DataLoaders(train,test,'resnet',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()

resnet_model = ResNet50(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,resnet_model,loss_func,optimizer,device,'resnet50')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])

pd.DataFrame(tr).to_csv("stl10_resnet50_tr.csv") 
pd.DataFrame(ts).to_csv("stl10_resnet50_ts.csv")


100%|██████████| 3/3 [00:47<00:00, 15.99s/it]

Final Train Accuracy: 0.9612
Final Test Accuracy: 0.95762503
Final Train Loss: 0.2038042620767521
Final Test Loss: 0.20766171097755431


In [ ]:
torch.save(resnet_model.state_dict(), "stl10_resnet50.pth")

In [ ]:
train = CustomDataset(train_images,train_labels,transforms_vit)
test = CustomDataset(test_images,test_labels,transforms_vit)

dl = DataLoaders(train,test,'resnet',BATCH_SIZE,True,'STL10')
train_loader, test_loader = dl.get_loaders()

vit_model = ViTBase(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vit_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,vit_model,loss_func,optimizer,device,'vit')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])


pd.DataFrame(tr).to_csv("stl10_vitbase_tr.csv") 
pd.DataFrame(ts).to_csv("stl10_vitbase_ts.csv")

100%|██████████| 3/3 [01:48<00:00, 36.12s/it]

Final Train Accuracy: 0.9992
Final Test Accuracy: 0.9955
Final Train Loss: 0.013579745930206926
Final Test Loss: 0.021977469671517612


In [ ]:
torch.save(vit_model.state_dict(), "stl10_vitbase.pth")

In [ ]:
train = CustomDataset(train_images,train_labels)
test = CustomDataset(test_images,test_labels)

dl = DataLoaders(train,test,'clip',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()

clip_model = CLIPModel(256)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(clip_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,clip_model,loss_func,optimizer,device,'clip')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])


pd.DataFrame(tr).to_csv("stl10_clip_tr.csv") 
pd.DataFrame(ts).to_csv("stl10_clip_ts.csv")

100%|██████████| 3/3 [01:02<00:00, 20.88s/it]

Final Train Accuracy: 0.98859996
Final Test Accuracy: 0.98925006
Final Train Loss: 0.6283798368134292
Final Test Loss: 0.22787716377199824


In [ ]:
torch.save(clip_model.state_dict(), "stl10_clip.pth")

## Evaluating Shape Bias

### Silhouettes

In [ ]:
train = CustomDataset(train_images,train_labels,transforms.Compose([Silhouette(),transforms_resnet]))
test = CustomDataset(test_images,test_labels,transforms.Compose([Silhouette(),transforms_resnet]))
BATCH_SIZE = 64
N_EPOCHS = 3
dl = DataLoaders(train,test,'resnet',BATCH_SIZE,True,'STL10')
train_loader, test_loader = dl.get_loaders()

vit_model = ResNet50(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vit_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,vit_model,loss_func,optimizer,device,'resnet50')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])


100%|██████████| 3/3 [00:51<00:00, 17.14s/it]

Final Train Accuracy: 0.5572
Final Test Accuracy: 0.50362504
Final Train Loss: 1.307312656052505
Final Test Loss: 1.3839964952468873


In [ ]:
train = CustomDataset(train_images,train_labels,transforms.Compose([Silhouette(),transforms_vit]))
test = CustomDataset(test_images,test_labels,transforms.Compose([Silhouette(),transforms_vit]))
BATCH_SIZE = 64
N_EPOCHS = 3
dl = DataLoaders(train,test,'vit',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()


dl = DataLoaders(train,test,'resnet',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()

vit_model = ViTBase(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vit_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,vit_model,loss_func,optimizer,device,'vit')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])


100%|██████████| 3/3 [01:53<00:00, 37.80s/it]

Final Train Accuracy: 0.2872
Final Test Accuracy: 0.28062502
Final Train Loss: 1.916887268235412
Final Test Loss: 1.9133425569534301


In [ ]:
train = CustomDataset(train_images,train_labels,Silhouette())
test = CustomDataset(test_images,test_labels,Silhouette())
BATCH_SIZE = 64
N_EPOCHS = 3
dl = DataLoaders(train,test,'clip',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()

clip_model = CLIPModel(256)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(clip_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,clip_model,loss_func,optimizer,device,'clip')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])

100%|██████████| 3/3 [00:57<00:00, 19.24s/it]

Final Train Accuracy: 0.4052
Final Test Accuracy: 0.45862502
Final Train Loss: 4.243603162050247
Final Test Loss: 1.9466484680175782


### Cleaner Silhouettes

In [ ]:
train = CustomDataset(train_images,train_labels,transforms.Compose([CleanerSilhouette(),transforms_resnet]))
test = CustomDataset(test_images,test_labels,transforms.Compose([CleanerSilhouette(),transforms_resnet]))
BATCH_SIZE = 64
N_EPOCHS = 3
dl = DataLoaders(train,test,'resnet',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()

vit_model = ResNet50(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vit_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,vit_model,loss_func,optimizer,device,'resnet50')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])


100%|██████████| 3/3 [01:08<00:00, 22.85s/it]

Final Train Accuracy: 0.473
Final Test Accuracy: 0.42025003
Final Train Loss: 1.522163572190683
Final Test Loss: 1.5924991607666015


In [ ]:
train = CustomDataset(train_images,train_labels,transforms.Compose([CleanerSilhouette(),transforms_vit]))
test = CustomDataset(test_images,test_labels,transforms.Compose([CleanerSilhouette(),transforms_vit]))
BATCH_SIZE = 64
N_EPOCHS = 3
dl = DataLoaders(train,test,'vit',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()


dl = DataLoaders(train,test,'resnet',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()

vit_model = ViTBase(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vit_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,vit_model,loss_func,optimizer,device,'vit')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])


100%|██████████| 3/3 [01:59<00:00, 39.92s/it]

Final Train Accuracy: 0.2572
Final Test Accuracy: 0.28525
Final Train Loss: 1.970516956305202
Final Test Loss: 1.9025903387069703


In [ ]:
train = CustomDataset(train_images,train_labels,CleanerSilhouette())
test = CustomDataset(test_images,test_labels,CleanerSilhouette())
BATCH_SIZE = 64
N_EPOCHS = 3
dl = DataLoaders(train,test,'clip',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()

clip_model = CLIPModel(256)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(clip_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,clip_model,loss_func,optimizer,device,'clip')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])

100%|██████████| 3/3 [00:59<00:00, 19.80s/it]

Final Train Accuracy: 0.3254
Final Test Accuracy: 0.34337503
Final Train Loss: 3.099897040605545
Final Test Loss: 2.047487508773804


## GrayScale

In [ ]:
train = CustomDataset(train_images,train_labels,transforms.Compose([Grayscale(),transforms_resnet]))
test = CustomDataset(test_images,test_labels,transforms.Compose([Grayscale(),transforms_resnet]))
BATCH_SIZE = 64
N_EPOCHS = 3
dl = DataLoaders(train,test,'resnet',BATCH_SIZE,True,'STL10')
train_loader, test_loader = dl.get_loaders()

vit_model = ResNet50(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vit_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,vit_model,loss_func,optimizer,device,'resnet50')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])


100%|██████████| 3/3 [00:50<00:00, 16.97s/it]

Final Train Accuracy: 0.921
Final Test Accuracy: 0.90987504
Final Train Loss: 0.33622876290656345
Final Test Loss: 0.34240135622024537


In [ ]:
train = CustomDataset(train_images,train_labels,transforms.Compose([Grayscale(),transforms_vit]))
test = CustomDataset(test_images,test_labels,transforms.Compose([Grayscale(),transforms_vit]))
BATCH_SIZE = 64
N_EPOCHS = 3
dl = DataLoaders(train,test,'vit',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()


dl = DataLoaders(train,test,'resnet',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()

vit_model = ViTBase(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vit_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,vit_model,loss_func,optimizer,device,'vit')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])


100%|██████████| 3/3 [01:53<00:00, 37.82s/it]

Final Train Accuracy: 0.99579996
Final Test Accuracy: 0.990625
Final Train Loss: 0.02246451720336004
Final Test Loss: 0.0368765226714313


In [ ]:
train = CustomDataset(train_images,train_labels,Grayscale())
test = CustomDataset(test_images,test_labels,Grayscale())
BATCH_SIZE = 64
N_EPOCHS = 3
dl = DataLoaders(train,test,'clip',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()

clip_model = CLIPModel(256)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(clip_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,clip_model,loss_func,optimizer,device,'clip')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])

100%|██████████| 3/3 [00:59<00:00, 19.90s/it]

Final Train Accuracy: 0.9738
Final Test Accuracy: 0.97162503
Final Train Loss: 0.7127005766789278
Final Test Loss: 0.642673109424776


### Outline

In [ ]:
train = CustomDataset(train_images,train_labels,transforms.Compose([Outline(),transforms_resnet]))
test = CustomDataset(test_images,test_labels,transforms.Compose([Outline(),transforms_resnet]))
BATCH_SIZE = 64
N_EPOCHS = 3
dl = DataLoaders(train,test,'resnet',BATCH_SIZE,True,'STL10')
train_loader, test_loader = dl.get_loaders()

vit_model = ResNet50(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vit_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,vit_model,loss_func,optimizer,device,'resnet50')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])


100%|██████████| 3/3 [01:28<00:00, 29.44s/it]

Final Train Accuracy: 0.5464
Final Test Accuracy: 0.49462503
Final Train Loss: 1.329365710668926
Final Test Loss: 1.3969348745346069


In [ ]:
train = CustomDataset(train_images,train_labels,transforms.Compose([Outline(),transforms_vit]))
test = CustomDataset(test_images,test_labels,transforms.Compose([Outline(),transforms_vit]))
BATCH_SIZE = 64
N_EPOCHS = 3
dl = DataLoaders(train,test,'vit',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()


dl = DataLoaders(train,test,'resnet',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()

vit_model = ViTBase(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vit_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,vit_model,loss_func,optimizer,device,'vit')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])


100%|██████████| 3/3 [03:01<00:00, 60.42s/it]

Final Train Accuracy: 0.6572
Final Test Accuracy: 0.64837503
Final Train Loss: 0.943323595614373
Final Test Loss: 0.9639347290992737


In [ ]:
train = CustomDataset(train_images,train_labels,Outline())
test = CustomDataset(test_images,test_labels,Outline())
BATCH_SIZE = 64
N_EPOCHS = 3
dl = DataLoaders(train,test,'clip',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()

clip_model = CLIPModel(256)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(clip_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,clip_model,loss_func,optimizer,device,'clip')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])

100%|██████████| 3/3 [00:58<00:00, 19.66s/it]

Final Train Accuracy: 0.5894
Final Test Accuracy: 0.610875
Final Train Loss: 3.163761470794678
Final Test Loss: 1.990285180091858


## Color Bias

#### Color Only (Blur)

In [ ]:
train = CustomDataset(train_images,train_labels,transforms.Compose([Blur(),transforms_resnet]))
test = CustomDataset(test_images,test_labels,transforms.Compose([Blur(),transforms_resnet]))
BATCH_SIZE = 64
N_EPOCHS = 5
dl = DataLoaders(train,test,'resnet',BATCH_SIZE,True,'STL10')
train_loader, test_loader = dl.get_loaders()

vit_model = ResNet50(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vit_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,vit_model,loss_func,optimizer,device,'resnet50')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])


100%|██████████| 5/5 [01:57<00:00, 23.49s/it]

Final Train Accuracy: 0.5652
Final Test Accuracy: 0.49050003
Final Train Loss: 1.280713460113429
Final Test Loss: 1.4199786157608032


In [ ]:
train = CustomDataset(train_images,train_labels,transforms.Compose([Blur(),transforms_vit]))
test = CustomDataset(test_images,test_labels,transforms.Compose([Blur(),transforms_vit]))
BATCH_SIZE = 64
N_EPOCHS = 5
dl = DataLoaders(train,test,'vit',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()


dl = DataLoaders(train,test,'resnet',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()

vit_model = ViTBase(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vit_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,vit_model,loss_func,optimizer,device,'vit')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])


100%|██████████| 5/5 [03:29<00:00, 41.84s/it]

Final Train Accuracy: 0.7104
Final Test Accuracy: 0.63525003
Final Train Loss: 0.8065641024444676
Final Test Loss: 1.0181063580513001


In [ ]:
train = CustomDataset(train_images,train_labels,Blur())
test = CustomDataset(test_images,test_labels,Blur())
BATCH_SIZE = 64
N_EPOCHS = 3
dl = DataLoaders(train,test,'clip',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()

clip_model = CLIPModel(256)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(clip_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'

tr,ts = train_and_eval(train_loader,test_loader,clip_model,loss_func,optimizer,device,'clip')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])

100%|██████████| 3/3 [01:00<00:00, 20.29s/it]

Final Train Accuracy: 0.5388
Final Test Accuracy: 0.525375
Final Train Loss: 3.8173225779533384
Final Test Loss: 2.4905688419342042


## Texture

### Random Patch Texture

In [ ]:
train = CustomDataset(train_images,train_labels,transforms.Compose([RandomPatchTexture(),transforms_resnet]))
test = CustomDataset(test_images,test_labels,transforms.Compose([RandomPatchTexture(),transforms_resnet]))
BATCH_SIZE = 64
N_EPOCHS = 3
dl = DataLoaders(train,test,'resnet',BATCH_SIZE,True,'STL10')
train_loader, test_loader = dl.get_loaders()

vit_model = ResNet50(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vit_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,vit_model,loss_func,optimizer,device,'resnet50')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])


100%|██████████| 3/3 [01:15<00:00, 25.01s/it]

Final Train Accuracy: 0.6806
Final Test Accuracy: 0.692375
Final Train Loss: 1.0060998389992533
Final Test Loss: 0.9605254735946656


In [ ]:
train = CustomDataset(train_images,train_labels,transforms.Compose([RandomPatchTexture(),transforms_vit]))
test = CustomDataset(test_images,test_labels,transforms.Compose([RandomPatchTexture(),transforms_vit]))
BATCH_SIZE = 64
N_EPOCHS = 3
dl = DataLoaders(train,test,'vit',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()


dl = DataLoaders(train,test,'resnet',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()

vit_model = ViTBase(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vit_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,vit_model,loss_func,optimizer,device,'vit')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])


100%|██████████| 3/3 [03:17<00:00, 65.86s/it]

Final Train Accuracy: 0.9666
Final Test Accuracy: 0.96000004
Final Train Loss: 0.10508971938368263
Final Test Loss: 0.12228052461147308


In [ ]:
train = CustomDataset(train_images,train_labels,RandomPatchTexture())
test = CustomDataset(test_images,test_labels,RandomPatchTexture())
BATCH_SIZE = 64
N_EPOCHS = 3
dl = DataLoaders(train,test,'clip',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()

clip_model = CLIPModel(256)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(clip_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,clip_model,loss_func,optimizer,device,'clip')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])

100%|██████████| 3/3 [01:40<00:00, 33.49s/it]

Final Train Accuracy: 0.8272
Final Test Accuracy: 0.88487506
Final Train Loss: 2.702853464268148
Final Test Loss: 0.7691263668537139


### Distorted Texture

In [ ]:
train = CustomDataset(train_images,train_labels,transforms.Compose([DistortedTexture(),transforms_resnet]))
test = CustomDataset(test_images,test_labels,transforms.Compose([DistortedTexture(),transforms_resnet]))
BATCH_SIZE = 64
N_EPOCHS = 3
dl = DataLoaders(train,test,'resnet',BATCH_SIZE,True,'STL10')
train_loader, test_loader = dl.get_loaders()

vit_model = ResNet50(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vit_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,vit_model,loss_func,optimizer,device,'resnet50')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])


100%|██████████| 3/3 [01:47<00:00, 35.97s/it]

Final Train Accuracy: 0.5704
Final Test Accuracy: 0.57412505
Final Train Loss: 1.2555050744286067
Final Test Loss: 1.2385056047439575


In [ ]:
train = CustomDataset(train_images,train_labels,transforms.Compose([DistortedTexture(),transforms_vit]))
test = CustomDataset(test_images,test_labels,transforms.Compose([DistortedTexture(),transforms_vit]))
BATCH_SIZE = 64
N_EPOCHS = 3
dl = DataLoaders(train,test,'vit',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()


dl = DataLoaders(train,test,'resnet',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()

vit_model = ViTBase(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vit_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,vit_model,loss_func,optimizer,device,'vit')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])


100%|██████████| 3/3 [03:32<00:00, 70.78s/it]

Final Train Accuracy: 0.7718
Final Test Accuracy: 0.75000006
Final Train Loss: 0.6663193001022821
Final Test Loss: 0.7112998473644256


In [ ]:
train = CustomDataset(train_images,train_labels,DistortedTexture())
test = CustomDataset(test_images,test_labels,DistortedTexture())
BATCH_SIZE = 64
N_EPOCHS = 3
dl = DataLoaders(train,test,'clip',BATCH_SIZE,True,'stl10')
train_loader, test_loader = dl.get_loaders()

clip_model = CLIPModel(256)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(clip_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,clip_model,loss_func,optimizer,device,'clip')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])

100%|██████████| 3/3 [01:01<00:00, 20.63s/it]

Final Train Accuracy: 0.5956
Final Test Accuracy: 0.59662503
Final Train Loss: 4.486887735009193
Final Test Loss: 2.711590178489685
